## Import Libraries

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from numpy import array
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

## Download Files

In [6]:
# # # Quinn
# path = 'C:\\Users\\Source\\Documents\\Descriptive and Predictive Analytics\\Group Project\\Project data\\'

In [7]:
# Leo
# path =''

In [8]:
# Mathieu
path = r'C:\Users\mperan\OneDrive - IESEG\Desktop\DP Analysis\Group Project\DSC_Project_Group2\Project data\\'

In [9]:
donors = pd.read_csv(path+'donors.csv', dtype={'zipcode': str})
campaigns = pd.read_csv(path+'campaigns.csv', sep=";", dtype={'campaignID': str})
gifts = pd.read_csv(path+'gifts.csv', dtype={'campaignID': str})
selection_campaign_6169 = pd.read_csv(path+'selection campaign 6169.csv')
selection_campaign_7244 = pd.read_csv(path+'selection campaign 7244.csv')

# Data Cleaning

# Campaigns

In [10]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   campaignID   367 non-null    object
 1   date         367 non-null    object
 2   lettersSent  367 non-null    object
 3   CostUnit     367 non-null    object
dtypes: object(4)
memory usage: 11.6+ KB


In [11]:
campaigns.head()

,campaignID,date,lettersSent,CostUnit
0,153,13/12/2004,6 873,"0,17 €"
1,154,01/01/2005,7 656,"0,30 €"
2,180,18/01/2005,9 933,"0,20 €"
3,433,29/01/2005,7 448,"0,59 €"
4,476,12/02/2005,6 605,"0,51 €"


In [12]:
campaigns['lettersSent'] = campaigns['lettersSent'].str.replace('\u202f', '').astype(int)

In [13]:
campaigns['CostUnit'] = campaigns['CostUnit'].str.replace('€', '').str.replace('\xa0', '')
campaigns['CostUnit'] = campaigns['CostUnit'].str.replace(',', '.')
campaigns['CostUnit'] = campaigns['CostUnit'].astype(float)

In [14]:
campaigns = campaigns.astype({'date': 'datetime64[ns]', 'lettersSent': 'int'})

In [15]:
campaigns.isnull().sum()/len(campaigns)

campaignID     0.0
date           0.0
lettersSent    0.0
CostUnit       0.0
dtype: float64

In [16]:
campaigns.dtypes

campaignID             object
date           datetime64[ns]
lettersSent             int32
CostUnit              float64
dtype: object

In [17]:
campaigns.head()

,campaignID,date,lettersSent,CostUnit
0,153,2004-12-13,6873,0.17
1,154,2005-01-01,7656,0.30
2,180,2005-01-18,9933,0.20
3,433,2005-01-29,7448,0.59
4,476,2005-12-02,6605,0.51


# Donors

In [18]:
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   donorID      44691 non-null  object
 1   zipcode      44689 non-null  object
 2   province     44691 non-null  object
 3   region       44691 non-null  object
 4   gender       44659 non-null  object
 5   language     44691 non-null  object
 6   dateOfBirth  44691 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


In [19]:
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth
0,D100001,1861,Flemish Brabant,Flanders,F,NL,12/24/1967
1,D100002,2260,Antwerp,Flanders,M,NL,01/14/1952
2,D100003,1780,Flemish Brabant,Flanders,M,NL,12/23/1986
3,D100004,1020,Brussels,Brussels,F,FR,03/29/1952
4,D100005,1020,Brussels,Brussels,F,FR,06/25/1986


In [20]:
donors = donors.astype({'dateOfBirth' : 'datetime64[ns]'})

In [21]:
donors['gender'].value_counts(normalize=True) * 100

M    57.750957
F    42.249043
Name: gender, dtype: float64

In [22]:
donors['gender'] = donors['gender'].fillna('M')
donors['zipcode'] = donors['zipcode'].fillna(0) 

In [23]:
donors.isnull().sum()

donorID        0
zipcode        0
province       0
region         0
gender         0
language       0
dateOfBirth    0
dtype: int64

In [24]:
def lowercase_except_first(value):
    return value[0] + value[1:].lower()

# Apply the function to the specified column
donors['province'] = donors['province'].apply(lowercase_except_first)

In [25]:
# Could potential drop after feature engineering
donors['language'].value_counts(normalize=True) * 100

NL    89.644447
FR    10.342127
EN     0.013426
Name: language, dtype: float64

In [26]:
donors['province'].value_counts(normalize=True) * 100

Antwerp            23.382784
East flanders      20.444832
West flanders      20.057730
Flemish brabant    15.173077
Limburg            10.780694
Brussels            3.329529
Liege               2.170459
Hainaut             2.112282
Walloon brabant     0.968875
Namur               0.897272
Luxembourg          0.677989
Missing             0.004475
Name: province, dtype: float64

In [27]:
# Combine everything with less than 10% to other
percentage = donors['province'].value_counts(normalize=True) * 100

# Find provinces that make up less than 10% of the dataset
small_provinces = percentage[percentage < 10].index.tolist()

# Replace 'province' values that are under 10% with 'Other'
donors['province'] = donors['province'].apply(lambda x: 'Other' if x in small_provinces else x)

In [28]:
donors['dateOfBirth'] = pd.to_datetime(donors['dateOfBirth'])
donors['Age'] = (datetime.now() - donors['dateOfBirth']).dt.days // 365

In [29]:
duplicates = donors.duplicated()

# To see if there are any duplicates in the DataFrame
any_duplicates = duplicates.any()
print(f"Are there any duplicate rows? {any_duplicates}")

if any_duplicates:
    print(donors[duplicates])

Are there any duplicate rows? False


In [30]:
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   donorID      44691 non-null  object        
 1   zipcode      44691 non-null  object        
 2   province     44691 non-null  object        
 3   region       44691 non-null  object        
 4   gender       44691 non-null  object        
 5   language     44691 non-null  object        
 6   dateOfBirth  44691 non-null  datetime64[ns]
 7   Age          44691 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.7+ MB


In [31]:
donors.isnull().sum()/len(donors)

donorID        0.0
zipcode        0.0
province       0.0
region         0.0
gender         0.0
language       0.0
dateOfBirth    0.0
Age            0.0
dtype: float64

In [32]:
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,Age
0,D100001,1861,Flemish brabant,Flanders,F,NL,1967-12-24,55
1,D100002,2260,Antwerp,Flanders,M,NL,1952-01-14,71
2,D100003,1780,Flemish brabant,Flanders,M,NL,1986-12-23,36
3,D100004,1020,Other,Brussels,F,FR,1952-03-29,71
4,D100005,1020,Other,Brussels,F,FR,1986-06-25,37


# Gifts

In [67]:
gifts[gifts['donorID']=='D100001'].size()

TypeError: 'numpy.int32' object is not callable

In [33]:
gifts.head()

,donorID,campaignID,amount,date
0,D100001,NaN,"12,39",23/03/2004
1,D100001,1577,"12,39",11/10/2007
2,D100001,1634,"6,69",28/12/2007
3,D100001,1778,"24,79",14/06/2008
4,D100001,1808,"7,44",09/07/2008


In [34]:
gifts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216594 entries, 0 to 216593
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   donorID     216594 non-null  object
 1   campaignID  200957 non-null  object
 2   amount      216594 non-null  object
 3   date        216594 non-null  object
dtypes: object(4)
memory usage: 6.6+ MB


In [35]:
#Fill null values in CampaignID to 0 (outside of campaign)
gifts['campaignID'] = gifts['campaignID'].fillna(0) # All donations outside of campaigns will be zero
# gifts['campaignID'] = gifts['campaignID'].astype(int)

In [36]:
gifts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216594 entries, 0 to 216593
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   donorID     216594 non-null  object
 1   campaignID  216594 non-null  object
 2   amount      216594 non-null  object
 3   date        216594 non-null  object
dtypes: object(4)
memory usage: 6.6+ MB


In [37]:
#Change date column to pd.datetime

gifts['date'] = pd.to_datetime(gifts['date'])

In [38]:
gifts.dtypes

donorID               object
campaignID            object
amount                object
date          datetime64[ns]
dtype: object

In [39]:
# Identify values with multiple commas --> INVALID DATA

multiple_commas = gifts[gifts['amount'].str.contains(r',.*,')]
print(multiple_commas)

        donorID campaignID                  amount       date
6360    D100778       1556  16,432,000,000,000,000 2007-10-09
16715   D101892          0   5,044,600,000,000,000 2000-11-22
28595   D103120          0  12,895,999,999,999,900 2006-03-24
30550   D103291        850   5,044,600,000,000,000 2005-10-15
33916   D103651       1933  11,595,999,999,999,900 2008-04-12
35879   D103890       1755  32,486,999,999,999,900 2008-12-05
49064   D105463          0  48,333,999,999,999,900 2003-11-24
53730   D106008          0  48,333,999,999,999,900 2006-04-05
56480   D106396          0  48,333,999,999,999,900 2001-03-20
56483   D106396          0  48,333,999,999,999,900 2002-05-31
56484   D106396          0  48,333,999,999,999,900 2002-10-22
63858   D107405          0  27,269,000,000,000,000 2006-07-11
78963   D109523          0  32,239,999,999,999,900 2005-03-28
90746   D111152          0  13,629,000,000,000,000 2003-08-18
92768   D111559       1778   6,445,269,999,999,990 2008-07-06
112525  

In [40]:
len(multiple_commas)/ len(gifts['amount']) *100

0.010618946046520217

In [41]:
gifts = gifts.drop(multiple_commas.index)

In [42]:
# transform amount column to correct format and float type

gifts['amount'] = gifts['amount'].str.replace(",",".")
gifts['amount'] = gifts['amount'].astype(float)

In [43]:
gifts['campaignID'].value_counts()

0       15621
4137     4020
1739     2824
3049     2022
2714     1946
        ...  
7356        1
574         1
1153        1
476         1
6924        1
Name: campaignID, Length: 368, dtype: int64

In [70]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Assuming 'date' is the name of your date column in the 'gifts' DataFrame
gifts['date'] = pd.to_datetime(gifts['date'])

# Define the time window for each dataset
time_window = relativedelta(years=5)

# Drop dates of campaigns
drop_train = pd.to_datetime('2018-09-04')
drop_test = pd.to_datetime('2019-06-18')
drop_score = pd.to_datetime('2020-02-01')

# Define the gap period: 2 weeks
gap_period = timedelta(days=14)

# Calculate the cutoff dates
cutoff_date_train = drop_train - gap_period
cutoff_date_test = drop_test - gap_period
cutoff_date_score = drop_score - gap_period

# Filter the datasets based on a fixed starting date: 5 years time window
gifts_train_start_period = cutoff_date_train - time_window
gifts_test_start_period = cutoff_date_test - time_window
gifts_score_start_period = cutoff_date_score - time_window

# Filter the datasets based on the conditions
gifts_train = gifts[(gifts['date'] >= gifts_train_start_period) & (gifts['date'] <= cutoff_date_train)]
gifts_test = gifts[(gifts['date'] >= gifts_test_start_period) & (gifts['date'] <= cutoff_date_test)]
gifts_score = gifts[(gifts['date'] >= gifts_score_start_period) & (gifts['date'] <= cutoff_date_score)]


In [72]:
gifts_train.sort_values(by='date')

,donorID,campaignID,amount,date
125230,D117527,3545,30.0,2013-08-21
156579,D124001,3545,50.0,2013-08-21
40000,D104418,3546,30.0,2013-08-21
29090,D103147,3545,15.0,2013-08-21
91822,D111366,3545,30.0,2013-08-21
...,...,...,...,...
96788,D112272,5449,50.0,2018-08-21
48795,D105428,5449,50.0,2018-08-21
1934,D100332,5449,41.0,2018-08-21
45573,D105067,5449,50.0,2018-08-21


In [68]:
gifts_train_start_period

Timestamp('2013-08-21 00:00:00')

In [45]:
cutoff_date_train

Timestamp('2018-08-21 00:00:00')

In [46]:
cutoff_date_test

Timestamp('2019-06-04 00:00:00')

In [47]:
cutoff_date_score

Timestamp('2020-01-18 00:00:00')

In [48]:
gifts_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42567 entries, 12 to 216590
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   donorID     42567 non-null  object        
 1   campaignID  42567 non-null  object        
 2   amount      42567 non-null  float64       
 3   date        42567 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.6+ MB


In [49]:
# Check if the time window is right (drop_date - 14 days)

gifts_train['date'].max()

Timestamp('2018-08-21 00:00:00')

In [50]:
# Check the time period of donations for the test campaign

gifts[gifts['campaignID']=='7244'].sort_values(by='date', ascending=True)

,donorID,campaignID,amount,date
128648,D118161,7244,50.0,2019-01-07
197726,D135588,7244,80.0,2019-01-07
178361,D129560,7244,100.0,2019-01-07
32927,D103550,7244,20.0,2019-01-07
205325,D138414,7244,4.0,2019-01-07
...,...,...,...,...
198997,D135983,7244,40.0,2019-08-07
208340,D140364,7244,80.0,2019-08-07
142519,D121011,7244,20.0,2019-08-07
115805,D115779,7244,5.0,2019-08-07


In [51]:
# Check the dates distribution of donations for our training campaign

gifts_test[gifts_test['campaignID']=='7244'].value_counts('date', sort=False)

date
2019-01-07    18
2019-02-07    20
2019-03-07    18
2019-04-07    19
2019-05-07    24
dtype: int64

In [52]:
gifts[gifts['campaignID']=='6169'].sort_values(by='date', ascending=True)

,donorID,campaignID,amount,date
48638,D105417,6169,6.0,2018-06-09
170162,D127370,6169,75.0,2018-06-09
198641,D135868,6169,120.0,2018-06-09
130506,D118609,6169,150.0,2018-06-09
121346,D116800,6169,30.0,2018-06-09
...,...,...,...,...
35331,D103826,6169,120.0,2018-12-09
71591,D108502,6169,120.0,2018-12-09
164995,D126070,6169,45.0,2018-12-09
131911,D118900,6169,123.0,2018-12-09


In [53]:
# Check if some donors have given twice or more for this campaign

duplicates_subset = gifts[gifts['campaignID'] == '6169'][gifts[gifts['campaignID'] == '6169'].duplicated(subset=['donorID'], keep=False)]
duplicates_subset

,donorID,campaignID,amount,date


In [54]:
gifts[gifts['campaignID']=='6169'].value_counts('date', sort=False, normalize=True) * 100

date
2018-06-09    5.205709
2018-07-09    4.701931
2018-08-09    5.541562
2018-09-09    5.121746
2018-09-13    5.541562
2018-09-14    4.785894
2018-09-15    5.793451
2018-09-16    5.373636
2018-09-17    5.037783
2018-09-18    5.541562
2018-09-19    5.037783
2018-09-20    5.373636
2018-09-21    6.549118
2018-09-22    5.205709
2018-09-23    5.961377
2018-09-24    4.869857
2018-10-09    4.785894
2018-11-09    4.534005
2018-12-09    5.037783
dtype: float64

In [55]:
# Check the dates distribution of donations for our training campaign

gifts_train[gifts_train['campaignID']=='6169'].value_counts('date', sort=False)

date
2018-06-09    62
2018-07-09    56
2018-08-09    66
dtype: int64

In [56]:
# Number of donations

gifts_train[gifts_train['campaignID']== '6169']

,donorID,campaignID,amount,date
6158,D100745,6169,30.0,2018-08-09
6410,D100778,6169,60.0,2018-07-09
6449,D100781,6169,15.0,2018-08-09
11749,D101344,6169,75.0,2018-07-09
11773,D101347,6169,120.0,2018-08-09
...,...,...,...,...
211376,D142168,6169,15.0,2018-08-09
211455,D142177,6169,30.0,2018-07-09
214780,D144356,6169,6.0,2018-06-09
215701,D144479,6169,60.0,2018-07-09


In [57]:
gifts_train.nunique()

donorID       20389
campaignID      141
amount          310
date           1765
dtype: int64

In [58]:
gifts_test.nunique()

donorID       19600
campaignID      143
amount          311
date           1752
dtype: int64

## Campaigns

In [59]:
selection_campaign_6169.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34888 entries, 0 to 34887
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   donorID  34888 non-null  object
dtypes: object(1)
memory usage: 272.7+ KB


In [60]:
selection_campaign_7244.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25645 entries, 0 to 25644
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   donorID  25645 non-null  object
dtypes: object(1)
memory usage: 200.5+ KB


In [61]:
donors.dtypes

donorID                object
zipcode                object
province               object
region                 object
gender                 object
language               object
dateOfBirth    datetime64[ns]
Age                     int64
dtype: object

In [62]:
# Save to a pickle file using pandas
donors.to_pickle('donors_cleaned.pkl')
gifts.to_pickle('gifts_cleaned.pkl')
gifts_train.to_pickle('gifts_trained_cleaned.pkl')
gifts_test.to_pickle('gifts_test_cleaned.pkl')
campaigns.to_pickle('campaigns_cleaned.pkl')